In [7]:
import pandas as pd
import datetime as dt

# Step 1: Load and combine the datasets with the correct encoding
try:
    df_2009_2010 = pd.read_csv('Year 2009--2010.csv', encoding='latin-1')
    df_2010_2011 = pd.read_csv('Year 2010--2011.csv', encoding='latin-1')
    df = pd.concat([df_2009_2010, df_2010_2011], ignore_index=True)
    print("Files loaded and combined successfully!")

except FileNotFoundError:
    print("Error: One or both of the CSV files could not be found.")
    print("Please make sure the files 'Year 2009--2010.csv' and 'Year 2010--2011.csv' are in the same directory as this script.")
    df = None # Set df to None to prevent further execution if files are not found

if df is not None:
    # Step 2: Clean the data
    # Drop rows where 'Customer ID' is missing
    df.dropna(subset=['Customer ID'], inplace=True)

    # Remove cancelled orders (invoices with a 'C')
    df = df[~df['Invoice'].astype(str).str.contains('C', na=False)]

    # Step 3: Prepare the columns
    # Calculate the total price for each line item
    df['TotalPrice'] = df['Quantity'] * df['Price']

    # Convert 'InvoiceDate' to datetime objects for accurate time calculations
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='mixed', dayfirst=False)

    print("\nData has been cleaned and prepared. Here's a preview:")
    print(df.head())

Files loaded and combined successfully!

Data has been cleaned and prepared. Here's a preview:
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country ï»¿Invoice  \
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom        NaN   
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom        NaN   
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom        NaN   
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom        NaN   
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom        NaN   

   TotalPrice  
0        83.4  
1        81.0  
2  

In [8]:
import pandas as pd

# This code assumes the 'df' DataFrame has been loaded and cleaned
# in the previous step and the 'InvoiceDate' is in datetime format.

# Step 1: Extract the year from the 'InvoiceDate'
df['InvoiceYear'] = df['InvoiceDate'].dt.year

# Step 2: Group the data by year and calculate the summary metrics
sales_summary = df.groupby('InvoiceYear').agg(
    TotalSales=('TotalPrice', 'sum'),
    TotalUniqueCustomers=('Customer ID', 'nunique')
).reset_index()

# Step 3: Print the summary table
print("Year-over-Year Sales Summary:")
print(sales_summary)

# Step 4: Export the summary to a new CSV file for Power BI
sales_summary.to_csv('sales_summary.csv', index=False)

print("\nYour sales summary has been saved to a new file named 'sales_summary.csv'.")

Year-over-Year Sales Summary:
   InvoiceYear   TotalSales  TotalUniqueCustomers
0         2009   686654.160                   955
1         2010  8699953.134                  4234
2         2011  7745479.794                  4244

Your sales summary has been saved to a new file named 'sales_summary.csv'.


In [14]:
import pandas as pd
import datetime as dt

# Step 1: Load and combine the datasets with the correct encoding
try:
    df_2009_2010 = pd.read_csv('Year 2009--2010.csv', encoding='latin-1')
    df_2010_2011 = pd.read_csv('Year 2010--2011.csv', encoding='latin-1')
    df = pd.concat([df_2009_2010, df_2010_2011], ignore_index=True)
    print("Data loaded and combined successfully.")

    # Step 2: Clean the data
    df.dropna(subset=['Customer ID'], inplace=True)
    df = df[~df['Invoice'].astype(str).str.contains('C', na=False)]
    df['TotalPrice'] = df['Quantity'] * df['Price']
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='mixed', dayfirst=False)

    # Step 3: Calculate RFM values
    snapshot_date = max(df['InvoiceDate']) + dt.timedelta(days=1)
    rfm_df = df.groupby('Customer ID').agg(
        Recency=('InvoiceDate', lambda x: (snapshot_date - x.max()).days),
        Frequency=('Invoice', 'nunique'),
        Monetary=('TotalPrice', 'sum')
    ).reset_index()
    print("RFM values have been calculated.")

    # Step 4: Assign RFM scores with a robust method
    def get_score(series, is_recency=False):
        unique_values = series.nunique()
        bins = min(unique_values, 4)
        labels = [i + 1 for i in range(bins)]
        if is_recency:
            labels = labels[::-1]
        
        # Use qcut if possible, otherwise use a direct binning approach
        try:
            return pd.qcut(series, q=4, labels=labels, duplicates='drop')
        except ValueError:
            return pd.cut(series, bins=bins, labels=labels, duplicates='drop')
    
    rfm_df['R_Score'] = get_score(rfm_df['Recency'], is_recency=True)
    rfm_df['F_Score'] = get_score(rfm_df['Frequency'])
    rfm_df['M_Score'] = get_score(rfm_df['Monetary'])
    print("RFM scores have been assigned.")

    # Step 5: Assign RFM segments
    def rfm_level(row):
        r_score = int(row['R_Score'])
        f_score = int(row['F_Score'])
        m_score = int(row['M_Score'])
        if r_score >= 4 and f_score >= 4 and m_score >= 4:
            return 'Champions'
        elif r_score >= 4 and f_score >= 4 and m_score >= 3:
            return 'Loyal Customers'
        elif r_score >= 3 and f_score >= 3 and m_score >= 3:
            return 'Potential Loyalists'
        elif r_score >= 2 and f_score >= 2 and m_score >= 2:
            return 'At Risk'
        else:
            return 'Others'
    rfm_df['RFM_Level'] = rfm_df.apply(rfm_level, axis=1)

    # Step 6: Export the final RFM table to a new CSV file
    rfm_df.to_csv('rfm_analysis.csv', index=False)
    print("\nYour RFM analysis has been saved to a new file named 'rfm_analysis.csv'.")

except FileNotFoundError:
    print("Error: The CSV files could not be found.")
    print("Please make sure the files 'Year 2009--2010.csv' and 'Year 2010--2011.csv' are in the same directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data loaded and combined successfully.
RFM values have been calculated.
RFM scores have been assigned.

Your RFM analysis has been saved to a new file named 'rfm_analysis.csv'.


In [2]:
import pandas as pd
import datetime as dt

# Step 1: Load and combine the datasets with the correct encoding
try:
    df_2009_2010 = pd.read_csv('Year 2009--2010.csv', encoding='latin-1')
    df_2010_2011 = pd.read_csv('Year 2010--2011.csv', encoding='latin-1')
    df = pd.concat([df_2009_2010, df_2010_2011], ignore_index=True)
    print("Data loaded and combined successfully.")

    # Step 2: Clean the data
    df.dropna(subset=['Customer ID', 'Country'], inplace=True)
    df = df[~df['Invoice'].astype(str).str.contains('C', na=False)]
    df['TotalPrice'] = df['Quantity'] * df['Price']
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='mixed', dayfirst=False)
    df['InvoiceYear'] = df['InvoiceDate'].dt.year

    # Step 3: Calculate the sales summary by year and country
    sales_summary_df = df.groupby(['InvoiceYear', 'Country']).agg(
        TotalSales=('TotalPrice', 'sum'),
        TotalUniqueCustomers=('Customer ID', 'nunique')
    ).reset_index()

    # Step 4: Export the new summary to a new CSV file
    sales_summary_df.to_csv('sales_summary.csv', index=False)
    print("\nYour updated sales summary has been saved to a new file named 'sales_summary.csv'.")

except FileNotFoundError:
    print("Error: The CSV files could not be found.")
    print("Please make sure the files 'Year 2009--2010.csv' and 'Year 2010--2011.csv' are in the same directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data loaded and combined successfully.

Your updated sales summary has been saved to a new file named 'sales_summary.csv'.
